# Structural Changes in the q Model: Lecture Notes Experiments

This notebook simulates the thought experiments discussed in Christopher D. Carroll's graduate
Macroeconomics [lecture notes](http://www.econ2.jhu.edu/people/ccarroll/public/lecturenotes/Investment/qModel/):
productivity, corporate tax rate, and investment tax credit changes. For each experiment, the figure from the lecture notes is reproduced.

For each change, I display the behavior of the model in two different contexts:
* **Unanticipated change**: The change takes place at $t=0$ without notice.
* **Announced change**: The change is announced at $t=0$ but takes effect at $t=5$.

In [ ]:

# ============================================================================
# Preamble: Import libraries and configure plotting
# ============================================================================

# Numerical computing and array operations
import numpy as np

# Plotting and visualization
import matplotlib
import matplotlib.pyplot as plt
import warnings

# Set matplotlib style (try newer version first, fallback to older)
try:
    plt.style.use("seaborn-v0_8-whitegrid")
except (OSError, ValueError):
    plt.style.use("seaborn-whitegrid")

# Configure matplotlib for publication-quality figures
matplotlib.rcParams["axes.labelsize"] = 18
matplotlib.rcParams.update({
    "axes.titlesize": 16,
    "axes.edgecolor": "#444444",
    "axes.spines.top": False,      # Remove top spine for cleaner look
    "axes.spines.right": False,    # Remove right spine for cleaner look
    "figure.facecolor": "#f7f7f7", # Light gray background
    "axes.facecolor": "#f7f7f7",   # Light gray axes background
    "legend.frameon": False,       # No frame around legend
})

# Suppress expected warnings from numerical optimization
# These occur when capital becomes negative or diverges during optimization
warnings.filterwarnings(
    "ignore",
    message=r"\s*Capital (became negative|diverged).*Holding capital constant.",
    category=RuntimeWarning,
)
warnings.filterwarnings(
    "ignore",
    category=RuntimeWarning,
    module=r".*Qmod\.q_investment",  # Suppress warnings from Qmod module
)

# Utilities for copying model objects
from copy import deepcopy

# Optimization routines (used for finding optimal investment path)
from scipy import optimize

# Dolo: Dynamic optimization library for economic models
# Used here for potential future extensions, though main analysis uses Qmod
from dolo import *
import dolo.algos.perfect_foresight as pf
import dolo.algos.value_iteration as vi

# Data manipulation (imported but not heavily used in this notebook)
import pandas as pd

# Import Qmod class from parent directory
# Qmod implements the Abel-Hayashi 'Marginal Q' model of investment
import sys
sys.path.append('../')
from Qmod import Qmod


## Function Definitions

Since the plots for every experiment have the same format, I first define functions that carry out the analysis given a path for the exogenous variables.

In [ ]:

# ============================================================================
# Function Definitions
# ============================================================================

def pathValue(invest, mod1, mod2, k0, t):
    """
    Compute the present discounted value of an investment path under structural change.
    
    This function calculates the total value (present discounted utility/profit) of
    following a specific investment path [i(0), i(1), ..., i(t-1)] when the model
    parameters change at time t. Before time t, model mod1 applies; from time t
    onwards, model mod2 applies.
    
    Parameters
    ----------
    invest : array-like
        Investment decisions for periods 0 to t-1. Shape: (t,)
    mod1 : Qmod
        Q-model object representing parameter values before the structural change
        (prevails from time 0 to t-1)
    mod2 : Qmod
        Q-model object representing parameter values after the structural change
        (prevails from time t onwards)
    k0 : float
        Initial capital stock at time 0
    t : int
        Time period when the structural change occurs (model switches from mod1 to mod2)
    
    Returns
    -------
    float
        Present discounted value of the investment path, computed as:
        - Sum of discounted flow values from periods 0 to t-1 (using mod1)
        - Plus discounted continuation value at time t (using mod2's value function)
    
    Notes
    -----
    The value function uses mod1's discount factor (beta) for all periods, ensuring
    consistent discounting. The continuation value at time t is computed using
    mod2's value function, which incorporates the new parameter values.
    """
    # Initialize capital path: k[0] = k0, k[1] through k[t] computed below
    k = np.zeros(t + 1)
    k[0] = k0
    
    # Initialize total present discounted value
    value = 0

    # Compute flow values and capital accumulation for periods 0 to t-1
    # During these periods, mod1 (pre-change model) applies
    for i in range(t):
        # Flow value: profit/utility in period i given capital k[i] and investment invest[i]
        flow = mod1.flow(k[i], invest[i])
        
        # Add discounted flow value to total (discounted by beta^i)
        value += flow * mod1.beta ** i
        
        # Update capital: k[i+1] = (1-delta)*k[i] + invest[i]
        k[i + 1] = k[i] * (1 - mod1.delta) + invest[i]

    # Add continuation value: from time t onwards, mod2 applies
    # The value at time t is computed using mod2's value function, discounted by beta^t
    value += (mod1.beta ** t) * mod2.value_func(k[t])
    
    return value

def structural_change(mod1, mod2, k0, t_change, T_sim, npoints=300, figname=None, experiment_label=None):
    """
    Simulate and visualize the optimal response to a structural change in the Q-model.
    
    This function computes the optimal capital (k) and shadow price (lambda) dynamics
    when model parameters change at time t_change. It handles both unanticipated changes
    (t_change=0) and announced changes (t_change>0), where agents can optimize their
    investment path in anticipation of the change.
    
    The function generates a comprehensive 6-panel figure showing:
    1. Phase diagram: k vs lambda (with loci and stable arms)
    2. Phase diagram: k vs q (Tobin's q)
    3. Capital dynamics over time
    4. Shadow price (lambda) dynamics over time
    5. Investment dynamics over time
    6. Tobin's q dynamics over time
    
    Parameters
    ----------
    mod1 : Qmod
        Q-model object with parameters before the structural change
        (applies from time 0 to t_change-1)
    mod2 : Qmod
        Q-model object with parameters after the structural change
        (applies from time t_change onwards)
    k0 : float
        Initial capital stock at time 0
    t_change : int
        Time period when structural change occurs
        - If t_change == 0: unanticipated change (no optimization before change)
        - If t_change > 0: announced change (optimal investment path computed)
    T_sim : int
        Total number of periods to simulate
    npoints : int, optional
        Number of points for plotting phase diagrams (default: 300)
    figname : str, optional
        Base filename for saving figures (saves as .svg, .png, .pdf)
        If None, figures are not saved (default: None)
    experiment_label : str, optional
        Custom label for the experiment. If None, automatically generated
        from parameter differences (default: None)
    
    Returns
    -------
    dict
        Dictionary with keys:
        - 'k': array of capital stock values over time
        - 'lambda': array of shadow price (lambda) values over time
    
    Notes
    -----
    For announced changes (t_change > 0), the function uses scipy.optimize to find
    the optimal investment path that maximizes the present discounted value, taking
    into account that the model parameters will change at time t_change.
    
    The function computes:
    - Optimal investment path (if announced)
    - Capital accumulation: k[t+1] = (1-delta)*k[t] + i[t]
    - Shadow price: lambda[t] = marginal value of capital
    - Tobin's q: q[t] = lambda[t] / (1 - zeta[t])
    
    See Also
    --------
    pathValue : Function used to compute value of investment paths
    """
    
    # ========================================================================
    # Helper function: Generate descriptive label for the experiment
    # ========================================================================
    def describe_experiment():
        if experiment_label:
            return experiment_label

        change_bits = []
        for attr, label in [
            ("psi", "Productivity (psi)"),
            ("tau", "Corporate tax (tau)"),
            ("zeta", "Investment tax credit (zeta)"),
            ("alpha", "Capital share (alpha)"),
            ("omega", "Adjustment cost (omega)"),
            ("delta", "Depreciation (delta)"),
            ("beta", "Discount factor (beta)"),
        ]:
            old, new = getattr(mod1, attr), getattr(mod2, attr)
            if not np.isclose(old, new):
                change_bits.append(f"{label}: {old:.3g} -> {new:.3g}")

        timing = (
            "Unanticipated: takes effect at t=0"
            if t_change == 0
            else f"Announced at t=0, takes effect at t={t_change}"
        )
        if not change_bits:
            change_bits.append("Structural change")
        return " | ".join(change_bits + [timing])

    # ========================================================================
    # Helper function: Compute axis limits with padding for better visualization
    # ========================================================================
    def padded_limits(arrs, pad=0.08, fallback=1.0):
        """
        Compute axis limits with padding from a collection of arrays.
        
        Takes multiple arrays, concatenates them, and computes min/max with
        a percentage-based padding buffer for cleaner plot appearance.
        """
        data = np.concatenate([np.ravel(np.asarray(a)) for a in arrs])
        data = data[np.isfinite(data)]  # Remove infinite/NaN values
        if data.size == 0:
            return (-fallback, fallback)
        low, high = data.min(), data.max()
        span = high - low
        if span == 0:
            span = max(abs(low), fallback)
        buffer = span * pad if span > 0 else pad * fallback
        return low - buffer, high + buffer

    # Generate title for the figure
    title_text = describe_experiment()

    # ========================================================================
    # Step 1: Compute optimal investment path (if change is announced)
    # ========================================================================
    if t_change > 0:
        # For announced changes, find optimal investment path that maximizes
        # present discounted value, accounting for the structural change at t_change
        fobj = lambda x: -1 * pathValue(x, mod1, mod2, k0, t_change)  # Negative for minimization
        inv = optimize.minimize(
            fobj,
            x0=np.ones(t_change) * mod1.kss * mod2.delta,  # Initial guess: steady-state investment
            options={'disp': True},
            tol=1e-16
        ).x
    else:
        # For unanticipated changes (t_change == 0), no pre-optimization needed
        inv = None

    # ========================================================================
    # Step 2: Simulate capital, shadow price, and investment dynamics
    # ========================================================================
    k = np.zeros(T_sim)      # Capital stock over time
    lam = np.zeros(T_sim)    # Shadow price (marginal value of capital) over time
    invest = np.zeros(T_sim)  # Investment over time

    k[0] = k0  # Set initial capital
    
    # Simulate dynamics period by period
    for i in range(0, T_sim - 1):
        if i < t_change:
            # Before structural change: use mod1 and optimal investment path
            k[i + 1] = k[i] * (1 - mod1.delta) + inv[i]  # Capital accumulation
            lam[i] = mod1.findLambda(k[i], k[i + 1])      # Shadow price from mod1
            invest[i] = inv[i]                             # Investment from optimal path
        else:
            # After structural change: use mod2's optimal policy function
            k[i + 1] = mod2.k1Func(k[i])                  # Optimal next-period capital
            lam[i] = mod2.findLambda(k[i], k[i + 1])       # Shadow price from mod2
            invest[i] = k[i + 1] - (1 - mod2.delta) * k[i]  # Implied investment

    # Compute final period values (using mod2 since change has occurred)
    lam[T_sim - 1] = mod2.findLambda(k[T_sim - 1], mod2.k1Func(k[T_sim - 1]))
    invest[T_sim - 1] = mod2.k1Func(k[T_sim - 1]) - (1 - mod2.delta) * k[T_sim - 1]

    # ========================================================================
    # Step 3: Compute Tobin's q
    # ========================================================================
    # Tobin's q = lambda / (1 - zeta), where zeta is investment tax credit
    # Pcal is the price of capital after tax credit adjustment
    Pcal = np.array([1 - mod1.zeta] * t_change + [1 - mod2.zeta] * (T_sim - t_change))
    q = lam / Pcal  # Tobin's q: market value relative to replacement cost

    fig, ax = plt.subplots(3, 2, figsize=(15,12), sharex='col',
                           gridspec_kw={'hspace':0.38, 'wspace':0.22})
    fig.patch.set_facecolor('#f7f7f7')

    path_color = '#2f4858'
    highlight_color = '#d62728'
    colors = ['#1f77b4','#ff7f0e']
    labels = ['Pre-change','Post-change']

    k_range = np.linspace(0.1*min(mod1.kss,mod2.kss),2*max(mod1.kss,mod2.kss),
                          npoints)
    mods = [mod1,mod2]
    lambda_loci = []
    stable_arms = []
    stable_arms_q = []

    # Plot k,lambda path (dots only)
    ax[0,0].scatter(k, lam, color=path_color, s=26, label='Transition path', zorder=3)
    ax[0,0].scatter(k[t_change],lam[t_change], color=highlight_color, s=36, zorder=5,
                    label='Change takes effect')

    for i in range(2):
        lam_locus = np.array([mods[i].lambda0locus(x) for x in k_range])
        lambda_loci.append(lam_locus)
        arm = np.array([mods[i].findLambda(k0=x, k1=mods[i].k1Func(x)) for x in k_range])
        stable_arms.append(arm)
        stable_arms_q.append(arm / mods[i].P)

        ax[0,0].plot(k_range,mods[i].P*np.ones(npoints),
                     linestyle = '--', color = colors[i],label = labels[i]+' $\dot{k}=0$')
        ax[0,0].plot(k_range,lam_locus,
                     linestyle = '--', color = colors[i], label = labels[i]+' $\dot{\lambda}=0$')
        ax[0,0].plot(k_range, arm, linestyle='-', color=colors[i], label=labels[i]+' stable arm')
        ax[0,0].plot(mods[i].kss,mods[i].P,marker = '*', markersize=12, color = colors[i])

    ax[0,0].set_xlabel('$k$')
    ax[0,0].set_ylabel('$\lambda$')
    ax[0,0].set_title('$k$ vs $\lambda$')
    ax[0,0].set_ylim(padded_limits([lam] + lambda_loci + [[mods[0].P, mods[1].P]]))
    ax[0,0].legend(fontsize=10)

    # 2nd plot: q phase diagrams (dots only)
    ax[0,1].scatter(k,q, color=path_color, s=26, label='Transition path', zorder=3)
    ax[0,1].scatter(k[t_change],q[t_change], color=highlight_color, s=36, zorder=5,
                    label='Change takes effect')

    q_loci = []
    for i in range(2):
        lam_locus = lambda_loci[i]
        q_locus = lam_locus/mods[i].P
        q_loci.append(q_locus)
        ax[0,1].plot(k_range,np.ones(npoints),
                     linestyle = '--', color = colors[i],label = labels[i]+' $\dot{k}=0$')
        ax[0,1].plot(k_range,q_locus,
                     linestyle = '--', color = colors[i], label = labels[i]+' $\dot{q}=0$')
        ax[0,1].plot(k_range, stable_arms_q[i], linestyle='-', color=colors[i], label=labels[i]+' stable arm')
        ax[0,1].plot(mods[i].kss,1,marker = '*', markersize=12, color = colors[i])

    ax[0,1].set_xlabel('$k$')
    ax[0,1].set_ylabel('$q$')
    ax[0,1].set_title('$k$ vs $q$')
    ax[0,1].set_ylim(padded_limits([q] + q_loci + [[1]]))
    ax[0,1].legend(fontsize=10)

    # 3rd plot: capital dynamics
    time = range(T_sim)
    for axis in ax[1:,:].ravel():
        axis.axvline(t_change, color=highlight_color, linestyle='--', linewidth=1,
                     alpha=0.8)

    ax[1,0].plot(time, k, color=path_color, marker='o', markersize=4, linestyle='None')
    ax[1,0].axhline(mod1.kss, color=colors[0], linestyle=':', linewidth=1.2,
                    label='Pre-change $k^*$')
    ax[1,0].axhline(mod2.kss, color=colors[1], linestyle='-.', linewidth=1.2,
                    label='Post-change $k^*$')
    ax[1,0].set_xlabel('$t$')
    ax[1,0].set_ylabel('$k_t$')
    ax[1,0].set_title('Capital dynamics')
    ax[1,0].set_ylim(padded_limits([k, [mod1.kss, mod2.kss]]))
    ax[1,0].legend(fontsize=11)

    # 4th plot: lambda dynamics
    ax[1,1].plot(time, lam, color=path_color, marker='o', markersize=4, linestyle='None')
    ax[1,1].axhline(mod1.P, color=colors[0], linestyle=':', linewidth=1.2,
                    label='Pre-change $\lambda^*$')
    ax[1,1].axhline(mod2.P, color=colors[1], linestyle='-.', linewidth=1.2,
                    label='Post-change $\lambda^*$')
    ax[1,1].set_xlabel('$t$')
    ax[1,1].set_ylabel('$\lambda_t$')
    ax[1,1].set_title('Marginal value dynamics')
    ax[1,1].set_ylim(padded_limits([lam, [mods[0].P, mods[1].P]]))
    ax[1,1].legend(fontsize=11)

    # 5th plot: investment dynamics
    ax[2,0].plot(time, invest, color=path_color, marker='o', markersize=4, linestyle='None')
    ax[2,0].axhline(0, color='#666666', linestyle=':', linewidth=1, alpha=0.7, label='$i=0$')
    ax[2,0].set_xlabel('$t$')
    ax[2,0].set_ylabel('$i_t$')
    ax[2,0].set_title('Investment dynamics')
    ax[2,0].set_ylim(padded_limits([invest, [0]]))
    ax[2,0].legend(fontsize=10)

    # 6th plot: q dynamics
    ax[2,1].plot(time, q, color=path_color, marker='o', markersize=4, linestyle='None')
    ax[2,1].axhline(1, color='#666666', linestyle=':', linewidth=1, alpha=0.7, label='$q=1$')
    ax[2,1].set_xlabel('$t$')
    ax[2,1].set_ylabel('$q_t$')
    ax[2,1].set_title('q dynamics')
    ax[2,1].set_ylim(padded_limits([q, [1]]))
    ax[2,1].legend(fontsize=10)

    for axis in ax.flat:
        axis.grid(True, alpha=0.25)

    fig.suptitle(title_text, y=0.965, fontsize=18)
    fig.tight_layout(rect=[0,0,1,0.95])

    if figname is not None:
        fig.savefig('../Figures/'+figname+'.svg', bbox_inches='tight', facecolor=fig.get_facecolor())
        fig.savefig('../Figures/'+figname+'.png', bbox_inches='tight', facecolor=fig.get_facecolor())
        fig.savefig('../Figures/'+figname+'.pdf', bbox_inches='tight', facecolor=fig.get_facecolor())

    return {'k':k, 'lambda':lam}


## Model Setup

We begin by defining the base parameters for the Q-model and initializing the model.

In [ ]:
# ============================================================================
# Base Model Parameters
# ============================================================================
# These parameters define the baseline Q-model. Experiments will modify
# specific parameters to study their effects on investment dynamics.

# Discount factor: how much future profits are valued relative to current
# beta = 0.98 means 1 unit tomorrow is worth 0.98 units today
beta = 0.98
R = 1 / beta  # Gross return factor (R = 1/beta ≈ 1.02)

# Corporate tax rate: fraction of profits paid as taxes
tau = 0.05

# Capital share in production function: Y = psi * k^alpha
# Higher alpha means capital is more important for production
alpha = 0.33

# Adjustment cost parameter: controls how costly it is to change investment
# Higher omega means larger adjustment costs (more inertia in investment)
omega = 1

# Investment tax credit: fraction of investment cost subsidized by government
# zeta = 0 means no subsidy; zeta > 0 reduces effective cost of investment
zeta = 0

# Depreciation rate: fraction of capital that wears out each period
delta = 0.1

# Productivity/technological factor: scales production function
# Higher psi means more output for given capital
psi = 1

# ============================================================================
# Initialize and Solve the Base Model
# ============================================================================
# Create Qmod instance with base parameters and solve for steady state
# The solve() method computes the value function and optimal policy functions
Qmodel = Qmod(beta, tau, alpha, omega, zeta, delta, psi)
Qmodel.solve()  # Computes steady state and value function


## Examples

### 1. An Unanticipated Increase in Productivity

In [ ]:
# ============================================================================
# Experiment 1: Unanticipated Productivity Increase
# ============================================================================
# This experiment studies what happens when productivity suddenly increases
# from psi=1.0 to psi=1.3 at time t=0, without any advance warning.

figname = 'ProductivityIncrease'
experiment_label = 'Unanticipated productivity increase (ψ 1.0 -> 1.3)'

# Simulation parameters
T = 20          # Total number of periods to simulate
t = 0           # Time when change occurs (0 = unanticipated, no optimization)
k0 = Qmodel.kss  # Start from steady state of baseline model

# New productivity level (30% increase)
psi_new = 1.3

# Create modified model with higher productivity
# deepcopy ensures we don't modify the original Qmodel
Q_high_psi = deepcopy(Qmodel)
Q_high_psi.psi = psi_new
Q_high_psi.solve()  # Re-solve for new steady state

# Run simulation and generate plots
sol = structural_change(
    mod1=Qmodel,        # Baseline model (before change)
    mod2=Q_high_psi,    # High productivity model (after change)
    k0=k0,
    t_change=t,
    T_sim=T,
    figname=figname,
    experiment_label=experiment_label
)
fig = plt.gcf()  # Get current figure for potential further manipulation

### 2. An Increase in Productivity Announced at t=0 but Taking Effect at t=5

In [ ]:
# ============================================================================
# Experiment 2: Announced Productivity Increase
# ============================================================================
# Same productivity increase as Experiment 1, but now announced at t=0
# and taking effect at t=5. Agents can optimize investment in anticipation.

figname = 'ProductivityIncrease-ant'
experiment_label = 'Productivity increase announced (ψ 1.0 -> 1.3), effect at t=5'

# Change timing: announced at t=0, takes effect at t=5
t = 5  # Agents have 5 periods to prepare

# Run simulation (Q_high_psi already created in Experiment 1)
sol = structural_change(
    mod1=Qmodel,
    mod2=Q_high_psi,
    k0=k0,
    t_change=t,
    T_sim=T,
    figname=figname,
    experiment_label=experiment_label
)

### 3. An Unanticipated Corporate Tax Cut

In [ ]:
# ============================================================================
# Experiment 3: Unanticipated Corporate Tax Cut
# ============================================================================
# Studies the effect of a sudden tax cut from tau=0.40 to tau=0.05.
# Note: We start from the high-tax steady state, then cut taxes.

figname = 'CorporateTaxReduction'
experiment_label = 'Unanticipated corporate tax cut (τ 0.40 -> 0.05)'

# Set high tax rate (baseline scenario)
tau_high = 0.4
t = 0  # Unanticipated change

# Create high-tax model and solve
Q_high_tau = deepcopy(Qmodel)
Q_high_tau.tau = tau_high
Q_high_tau.solve()

# Start from steady state of high-tax scenario
k0 = Q_high_tau.kss

# Run simulation: transition from high tax to low tax
sol = structural_change(
    mod1=Q_high_tau,    # High tax model (before change)
    mod2=Qmodel,        # Low tax model (after change, tau=0.05 from base)
    k0=k0,
    t_change=t,
    T_sim=T,
    figname=figname,
    experiment_label=experiment_label
)

### 4. A Corporate Tax Cut Announced at t=0 but Taking Effect at t=5

In [ ]:
# ============================================================================
# Experiment 4: Announced Corporate Tax Cut
# ============================================================================
# Same tax cut as Experiment 3, but announced in advance (takes effect at t=5).

figname = 'CorporateTaxReduction-ant'
experiment_label = 'Corporate tax cut announced (τ 0.40 -> 0.05), effect at t=5'

# Change timing: announced at t=0, takes effect at t=5
t = 5

# Run simulation (Q_high_tau already created in Experiment 3)
sol = structural_change(
    mod1=Q_high_tau,
    mod2=Qmodel,
    k0=k0,
    t_change=t,
    T_sim=T,
    figname=figname,
    experiment_label=experiment_label
)

### 5. An Unanticipated ITC Increase

In [ ]:
# ============================================================================
# Experiment 5: Unanticipated Investment Tax Credit (ITC) Increase
# ============================================================================
# Studies the effect of introducing an investment tax credit (zeta: 0 -> 0.20).
# ITC reduces the effective cost of investment by subsidizing a fraction.

figname = 'ITCIncrease'
experiment_label = 'Unanticipated ITC increase (ζ 0.00 -> 0.20)'

# Simulation parameters
t = 0           # Unanticipated change
itc_high = 0.2  # 20% investment tax credit
k0 = Qmodel.kss  # Start from baseline steady state

# Create model with ITC
Q_high_itc = deepcopy(Qmodel)
Q_high_itc.zeta = itc_high
Q_high_itc.solve()  # Re-solve for new steady state

# Run simulation
sol = structural_change(
    mod1=Qmodel,        # Baseline model (no ITC)
    mod2=Q_high_itc,    # Model with ITC
    k0=k0,
    t_change=t,
    T_sim=T,
    figname=figname,
    experiment_label=experiment_label
)

### 6. An ITC Increase Announced at t=0 but Taking Effect at t=5

In [ ]:
# ============================================================================
# Experiment 6: Announced Investment Tax Credit Increase
# ============================================================================
# Same ITC increase as Experiment 5, but announced in advance (takes effect at t=5).

figname = 'ITCIncrease-ant'
experiment_label = 'ITC increase announced (ζ 0.00 -> 0.20), effect at t=5'

# Change timing: announced at t=0, takes effect at t=5
t = 5

# Run simulation (Q_high_itc already created in Experiment 5)
sol = structural_change(
    mod1=Qmodel,
    mod2=Q_high_itc,
    k0=k0,
    t_change=t,
    T_sim=T,
    figname=figname,
    experiment_label=experiment_label
)